In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://edition.cnn.com/2024/02/06/investing/china-stocks-rally-government-stimulus/index.html'

In [ ]:
response = requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')
text = soup.get_text()

In [ ]:
text

" \n\n\n\n\n\n\n\n\nChina is pumping money into stocks and markets are loving it | CNN Business\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCNN values your feedback\n\n\n\n\n                                                        1. How relevant is this ad to you?\n                                                \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                                2. Did you encounter any technical issues?\n                                        \n\n\n\n\n\n\n\n\n\n\n\n                                                                        Video player was slow to load content\n                                                                        \n\n\n\n                                                                        Video content never loaded\n                  

In [ ]:
from transformers import BertTokenizer,BertForSequenceClassification

In [ ]:
from transformers import pipeline

In [ ]:
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
tokens = tokenizer(text, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/464k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2688 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis", num_labels=3)

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
# Tokenize the text using the FinancialBERT tokenizer with truncation
tokens = tokenizer(text, truncation=True, return_tensors="pt", max_length=512)

# Ensure the input tensor size matches the expected size
tokens['input_ids'] = tokens['input_ids'][:, :512]

# Forward pass
outputs = model(**tokens)
logits = outputs.logits

In [ ]:
import torch
import numpy as np

# Assuming 'logits' is the tensor obtained from the model
logits = outputs.logits

# Convert logits to probabilities using softmax
probs = torch.nn.functional.softmax(logits, dim=-1)

# Get the predicted sentiment label
predicted_label = torch.argmax(probs, dim=-1).item()

# Optionally, get the confidence score for the predicted label
confidence_score = probs[0, predicted_label].item()

# Print the results
print("Predicted Sentiment Label:", predicted_label)
print("Confidence Score:", confidence_score)


Predicted Sentiment Label: 1
Confidence Score: 0.9807201027870178


In [ ]:
import requests
from bs4 import BeautifulSoup
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Function to perform sentiment analysis on a given sentence
def analyze_sentiment(sentence, model, tokenizer):
    tokens = tokenizer(sentence, truncation=True, return_tensors="pt", max_length=512)
    tokens['input_ids'] = tokens['input_ids'][:, :512]

    outputs = model(**tokens)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)

    predicted_label = torch.argmax(probs, dim=-1).item()
    sentiment_text = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}[predicted_label]
    confidence_score = probs[0, predicted_label].item()

    return {'label': sentiment_text, 'score': confidence_score}

# Load the FinancialBERT model and tokenizer
model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

# Specify the URL
url = 'https://edition.cnn.com/2024/02/06/investing/china-stocks-rally-government-stimulus/index.html'

# Make a request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract text from all paragraphs
paragraphs = soup.find_all('p')
text = ' '.join([paragraph.get_text() for paragraph in paragraphs])

# Split the text into sentences
sentences = text.split('.')

# Initialize an empty list to store the sentiment results for each sentence
sentiment_results = []

# Perform sentiment analysis for each sentence
for i, sentence in enumerate(sentences, start=1):  # Starting index from 1
    # Skip empty sentences
    if not sentence.strip():
        continue

    # Analyze sentiment for the current sentence
    sentiment_result = analyze_sentiment(sentence, model, tokenizer)

    # Append the sentiment result for this sentence to the list
    sentiment_results.append({
        'sentence_number': i,
        'sentence': sentence,
        'sentiment': sentiment_result
    })

# Print the sentiment results for each sentence
for result in sentiment_results:
    print(f"Sentence {result['sentence_number']}: {result['sentence']}")
    print(f"Sentiment: {result['sentiment']}")
    print("\n")


Sentence 1: Markets 


 Hot Stocks 


 Fear & Greed Index 


 
            Latest Market News 


 
            Hot Stocks 


 
  Editor’s Note: Sign up for CNN’s Meanwhile in China newsletter, which explores what you need to know about the country’s rise and how it impacts the world
Sentiment: {'label': 'Neutral', 'score': 0.9997771382331848}


Sentence 2: 
 
            Chinese stocks staged their biggest rally in years Tuesday, after the country’s sovereign wealth fund said it would step up buying shares as officials scramble to draw a line under a three-year market rout
Sentiment: {'label': 'Positive', 'score': 0.9980061650276184}


Sentence 3: 
     
            The Shanghai Composite, mainland China’s benchmark index for large state-owned companies and blue-chip stocks, ended up 3
Sentiment: {'label': 'Positive', 'score': 0.9996092915534973}


Sentence 4: 2% on Tuesday, ending a six-day losing streak
Sentiment: {'label': 'Negative', 'score': 0.9931215643882751}


Sentence 5:  It m

In [ ]:
# Initialize lists to store positive, negative, and neutral sentences
positive_sentences = []
negative_sentences = []
neutral_sentences = []

# Iterate through sentiment results
for result in sentiment_results:
    sentence_number = result['sentence_number']
    sentence = result['sentence']
    sentiment_label = result['sentiment']['label']

    # Group sentences based on sentiment labels
    if sentiment_label == 'Positive':
        positive_sentences.append(sentence)
    elif sentiment_label == 'Negative':
        negative_sentences.append(sentence)
    else:
        neutral_sentences.append(sentence)

# Function to print grouped sentences and count
def print_grouped_sentences(sentences, sentiment_type):
    print(f"{sentiment_type} Sentences:")
    for i, sentence in enumerate(sentences, start=1):
        print(f"    {i}. {sentence.strip()}")
    print(f"    Count: {len(sentences)}")
    print("\n")

# Print grouped positive sentences and count
print_grouped_sentences(positive_sentences, "Positive")

# Print grouped negative sentences and count
print_grouped_sentences(negative_sentences, "Negative")

# Print grouped neutral sentences and count
print_grouped_sentences(neutral_sentences, "Neutral")


Positive Sentences:
    1. Chinese stocks staged their biggest rally in years Tuesday, after the country’s sovereign wealth fund said it would step up buying shares as officials scramble to draw a line under a three-year market rout
    2. The Shanghai Composite, mainland China’s benchmark index for large state-owned companies and blue-chip stocks, ended up 3
    3. It marks the biggest daily increase since March 2022
    4. Smaller companies did even better
    5. The Shenzhen Component Index rose 6
    6. It was the index’s best daily gain since September 2015, while startup index ChiNext had its best performance in seven years, soaring 6
    7. In Hong Kong, the Hang Seng Index surged 4%, the biggest percentage gain in more than six months
    8. Chinese tech shares led the way
    9. Alibaba Group (BABA), which is due to announce earnings on Wednesday, was up 7
    10. The positive mood spilled over to Europe, where the region’s benchmark Stoxx 600 index opened about 0
    11. 5% h

In [ ]:
# Initialize counters for positive, negative, and neutral reviews
positive_count = 0
negative_count = 0
neutral_count = 0

# Iterate through sentiment results
for result in sentiment_results:
    sentiment_label = result['sentiment']['label']

    # Update the counts based on sentiment labels
    if sentiment_label == 'Positive':
        positive_count += 1
    elif sentiment_label == 'Negative':
        negative_count += 1
    else:
        neutral_count += 1

# Calculate the collective sentiment
if positive_count > negative_count and positive_count > neutral_count:
    collective_sentiment = 'Positive'
elif negative_count > positive_count and negative_count > neutral_count:
    collective_sentiment = 'Negative'
else:
    collective_sentiment = 'Neutral'

# Print the collective sentiment and counts
print("Collective Sentiment:")
print(f"Positive Count: {positive_count}")
print(f"Negative Count: {negative_count}")
print(f"Neutral Count: {neutral_count}")
print(f"Collective Sentiment: {collective_sentiment}")


Collective Sentiment:
Positive Count: 16
Negative Count: 7
Neutral Count: 36
Collective Sentiment: Neutral


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the Bart model and tokenizer for summarization
model_summarization = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer_summarization = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Tokenize and summarize the text
input_ids = tokenizer_summarization.encode(text, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model_summarization.generate(input_ids, max_length=150, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer_summarization.decode(summary_ids[0], skip_special_tokens=True)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Print the summarized text
print("Summary:", summary)

Summary: Chinese stocks staged their biggest rally in years Tuesday, ending a six-day losing streak. China's sovereign wealth fund said it would step up buying shares as officials scramble to draw a line under a three-year market rout. The rebound in Chinese stocks came after Beijing stepped up its efforts to prop up the ailing stock market.
